In [ ]:
%pip install -U bitsandbytes transformers peft datasets trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from huggingface_hub import snapshot_download
import os
import shutil
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils.prune as prune
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [ ]:
from huggingface_hub import interpreter_login
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Enter your token (input will not be visible): ··········
Add token as git credential? (Y/n) y


In [ ]:
# Download LiteGuard Merged model.
adapter_path = "./Lora-Adapter/"
snapshot_download(repo_id="T-tchi0/Liteguard-LORA-adapter", local_dir=adapter_path)

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/62.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.86k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

...

adapter_config.json:   0%|          | 0.00/883 [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/506k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

'/content/Lora-Adapter'

In [ ]:
# Check if CUDA is available.
torch.cuda.is_available()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device


device(type='cuda', index=0)

In [ ]:

base_model_id = "microsoft/phi-2"
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

## Model Size reduction techniques

### Magnitude Pruning

In [ ]:
# Configuration
model_name = base_model
sparsity_target = 0.3  # 30% sparsity (prune 30% of weights)
output_dir = "./pruned_phi_2"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define pruning function for nn.Linear layers
def prune_linear_layers(model, sparsity):
    """
    Apply magnitude-based pruning to all nn.Linear layers in the model.
    Args:
        model: The model to prune (e.g., Phi-1.5).
        sparsity: Fraction of weights to prune (e.g., 0.3 for 30%).
    """
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear):
            print(f"Pruning layer: {name}")
            # Apply unstructured, magnitude-based pruning to the weight tensor
            prune.l1_unstructured(
                module,
                name="weight",
                amount=sparsity
            )
            # Remove the pruning reparametrization to make pruning permanent
            prune.remove(module, "weight")

# Apply pruning
print(f"Applying {sparsity_target*100}% sparsity to nn.Linear layers...")
prune_linear_layers(base_model, sparsity_target)

# Verify sparsity
def compute_sparsity(model):
    """
    Compute the percentage of zero weights in the model.
    """
    total_params = 0
    zero_params = 0
    for name, param in model.named_parameters():
        if "weight" in name:
            total_params += param.numel()
            zero_params += torch.sum(param == 0).item()
    sparsity = zero_params / total_params * 100
    return sparsity

sparsity = compute_sparsity(base_model)
print(f"Model sparsity after pruning: {sparsity:.2f}%")

# Save the pruned model
print(f"Saving pruned model to {output_dir}...")
os.makedirs(output_dir, exist_ok=True)
base_model.save_pretrained(output_dir)
# Check model size
def get_model_size(directory):
    """
    Compute the total size of model files in the directory (in MB).
    """
    total_size = 0
    for dirpath, _, filenames in os.walk(directory):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size / (1024 * 1024)  # Convert to MB

model_size = get_model_size(output_dir)
print(f"Pruned model size: {model_size:.2f} MB")

print("Pruning complete! The pruned model is ready for adapter integration and MLC LLM quantization.")

Applying 30.0% sparsity to nn.Linear layers...
Pruning layer: model.layers.0.self_attn.q_proj
Pruning layer: model.layers.0.self_attn.k_proj
Pruning layer: model.layers.0.self_attn.v_proj
Pruning layer: model.layers.0.self_attn.dense
Pruning layer: model.layers.0.mlp.fc1
Pruning layer: model.layers.0.mlp.fc2
Pruning layer: model.layers.1.self_attn.q_proj
Pruning layer: model.layers.1.self_attn.k_proj
Pruning layer: model.layers.1.self_attn.v_proj
Pruning layer: model.layers.1.self_attn.dense
Pruning layer: model.layers.1.mlp.fc1
Pruning layer: model.layers.1.mlp.fc2
Pruning layer: model.layers.2.self_attn.q_proj
Pruning layer: model.layers.2.self_attn.k_proj
Pruning layer: model.layers.2.self_attn.v_proj
Pruning layer: model.layers.2.self_attn.dense
Pruning layer: model.layers.2.mlp.fc1
Pruning layer: model.layers.2.mlp.fc2
Pruning layer: model.layers.3.self_attn.q_proj
Pruning layer: model.layers.3.self_attn.k_proj
Pruning layer: model.layers.3.self_attn.v_proj
Pruning layer: model.la

### Checking for parameters similarities and redundancy to guide Low-rank factorisation, Parameter Sharing and Weight clustering

In [ ]:
def check_weight_similarity(model):
    """
    Check cosine similarity of weights across layers for relevant layers in Phi-1.5.
    Args:
        model: Phi-1.5 model (already loaded)
    Returns:
        Dict with average similarity per layer.
    """
    layers = ["mlp.fc1", "mlp.fc2", "self_attn.dense", "self_attn.q_proj", "self_attn.k_proj", "self_attn.v_proj"]
    similarities = {}

    for layer_name in layers:
        weights = [module.weight for name, module in model.named_modules() if name.endswith(layer_name)]
        if not weights:
            print(f"No weights found for {layer_name}")
            similarities[layer_name] = 0
            continue
        layer_sims = []
        for i in range(1, len(weights)):
            w1 = weights[0].flatten()
            w2 = weights[i].flatten()
            cos_sim = F.cosine_similarity(w1, w2, dim=0).item()
            layer_sims.append(cos_sim)
        avg_sim = sum(layer_sims) / len(layer_sims) if layer_sims else 0
        print(f"Average cosine similarity for {layer_name}: {avg_sim:.4f}")
        similarities[layer_name] = avg_sim

    return similarities

def check_layer_redundancy(model):
    """
    Check cosine similarity of layer outputs for relevant layers in Phi-1.5.
    Args:
        model: Phi-1.5 model (already loaded)
    Returns:
        Dict with average redundancy per layer.
    """
    model.eval()
    layers = ["mlp.fc1", "mlp.fc2", "self_attn.dense", "self_attn.q_proj", "self_attn.k_proj", "self_attn.v_proj"]
    redundancies = {}

    for layer_name in layers:
        try:
            outputs = []
            first_layer = next(module for name, module in model.named_modules() if name.endswith(layer_name))
            dtype = first_layer.weight.dtype

            # Set input size based on Phi-1.5 architecture
            if layer_name == "mlp.fc1":
                input_size = 2048  # mlp.fc1: [2048] -> [8192]
            elif layer_name == "mlp.fc2":
                input_size = 8192  # mlp.fc2: [8192] -> [2048]
            else:
                input_size = 2048  # self_attn.*: [2048] -> [2048]

            # Phi-1.5 has 24 layers
            for i in range(24):
                if layer_name.startswith("self_attn"):
                    layer = getattr(model.model.layers[i].self_attn, layer_name.split('.')[-1])
                else:
                    layer = getattr(model.model.layers[i].mlp, layer_name.split('.')[-1])
                with torch.no_grad():
                    hidden_states = torch.rand(1, 10, input_size, dtype=dtype, device=model.device)
                    output = layer(hidden_states)
                    outputs.append(output.flatten())

            layer_sims = []
            for j in range(1, len(outputs)):
                cos_sim = F.cosine_similarity(outputs[0], outputs[j], dim=0).item()
                layer_sims.append(cos_sim)
            avg_sim = sum(layer_sims) / len(layer_sims) if layer_sims else 0
            print(f"Average output similarity for {layer_name}: {avg_sim:.4f}")
            redundancies[layer_name] = avg_sim
        except Exception as e:
            print(f"Error processing {layer_name}: {str(e)}")
            redundancies[layer_name] = 0

    return redundancies

# Modified function to handle both model paths and already loaded models
def analyze_phi_redundancy(model):
    """
    Analyze weight similarity and layer redundancy in a Phi-1.5 model.

    Args:
        model: Either a loaded model instance or a path/model_id to load from
    """
    from transformers import AutoModelForCausalLM

    # Check if model is a string (path or model_id) or an already loaded model
    if isinstance(model, str):
        print(f"Loading model from {model}...")
        try:
            model = AutoModelForCausalLM.from_pretrained(
                model,
                torch_dtype=torch.float16,
                device_map="auto"
            )
        except Exception as e:
            print(f"Error loading model: {str(e)}")
            return None
    else:
        print("Using provided model object...")

    print("\n--- Weight Similarity Analysis ---")
    weight_similarities = check_weight_similarity(model)

    print("\n--- Layer Redundancy Analysis ---")
    layer_redundancies = check_layer_redundancy(model)

    # Combined results
    print("\n--- Combined Results ---")
    for layer in weight_similarities.keys():
        w_sim = weight_similarities[layer]
        l_red = layer_redundancies[layer]
        print(f"{layer}: Weight Similarity = {w_sim:.4f}, Output Similarity = {l_red:.4f}")

    return {
        "weight_similarities": weight_similarities,
        "layer_redundancies": layer_redundancies
    }


In [ ]:
analyze_phi_redundancy(base_model)

Using provided model object...

--- Weight Similarity Analysis ---
Average cosine similarity for mlp.fc1: 0.0019
Average cosine similarity for mlp.fc2: -0.0000
Average cosine similarity for self_attn.dense: -0.0001
Average cosine similarity for self_attn.q_proj: 0.0001
Average cosine similarity for self_attn.k_proj: 0.0001
Average cosine similarity for self_attn.v_proj: 0.0000

--- Layer Redundancy Analysis ---
Error processing mlp.fc1: mat1 and mat2 shapes cannot be multiplied (10x2048 and 2560x10240)
Error processing mlp.fc2: mat1 and mat2 shapes cannot be multiplied (10x8192 and 10240x2560)
Error processing self_attn.dense: mat1 and mat2 shapes cannot be multiplied (10x2048 and 2560x2560)
Error processing self_attn.q_proj: mat1 and mat2 shapes cannot be multiplied (10x2048 and 2560x2560)
Error processing self_attn.k_proj: mat1 and mat2 shapes cannot be multiplied (10x2048 and 2560x2560)
Error processing self_attn.v_proj: mat1 and mat2 shapes cannot be multiplied (10x2048 and 2560x25

{'weight_similarities': {'mlp.fc1': 0.0018535275613107988,
  'mlp.fc2': -5.631677566036101e-06,
  'self_attn.dense': -6.887989659463205e-05,
  'self_attn.q_proj': 0.00011846903831728043,
  'self_attn.k_proj': 8.699009495396768e-05,
  'self_attn.v_proj': 1.2609266465710056e-05},
 'layer_redundancies': {'mlp.fc1': 0,
  'mlp.fc2': 0,
  'self_attn.dense': 0,
  'self_attn.q_proj': 0,
  'self_attn.k_proj': 0,
  'self_attn.v_proj': 0}}

##### Analysis before adding adapter

This analysis evaluates the eligibility of parameter sharing in a pruned Phi-2 model (28.81% sparsity, \~5.36 GB) by examining **weight similarity** and **layer redundancy** across six layers (`mlp.fc1`, `mlp.fc2`, `self_attn.dense`, `self_attn.q_proj`, `self_attn.k_proj`, `self_attn.v_proj`) before and after integrating a thrice fine-tuned LoRA adapter on `self_attn.q_proj`, `self_attn.k_proj`, and `self_attn.v_proj`. Weight similarity is measured via cosine similarity of weights across 32 layers, and layer redundancy is assessed by cosine similarity of outputs for synthetic inputs. Eligibility for parameter sharing is determined by thresholds: similarity or redundancy &gt;0.5 for safe sharing, &gt;0.3 for cautious sharing with fine-tuning, and &lt;0.3 as unsuitable.

##### Before LoRA Adapter Integration

- **Weight Similarity**:

  - `mlp.fc1`: 0.0019
  - `mlp.fc2`: -0.0000
  - `self_attn.dense`: -0.0001
  - `self_attn.q_proj`: 0.0001
  - `self_attn.k_proj`: 0 ď.0001
  - `self_attn.v_proj`: 0.0000
  - **Observation**: Near-zero similarities (range: -0.0001 to 0.0019) indicate highly dissimilar weights across layers, suggesting minimal structural redundancy. All layers fall well below the 0.3 threshold, rendering parameter sharing unsuitable based on weights alone.

- **Layer Redundancy**:

  - `mlp.fc1`: 0.3286
  - `mlp.fc2`: -0.0204
  - `self_attn.dense`: -0.0029
  - `self_attn.q_proj`: 0.0060
  - `self_attn.k_proj`: -0.0005
  - `self_attn.v_proj`: 0.0004
  - **Observation**: Only `mlp.fc1` (0.3286) exceeds the 0.3 threshold, indicating moderate output similarity and potential for cautious sharing with fine-tuning. Other layers show negligible or negative redundancy (range: -0.0204 to 0.0060), suggesting distinct functional roles and unsuitability for sharing.

- **Conclusion**: Parameter sharing is not recommended except for `mlp.fc1`, where moderate redundancy supports cautious sharing with fine-tuning. Low weight similarity across all layers warns of significant accuracy risks if shared without mitigation.


In [ ]:
from peft import PeftModel

# Load the pruned model
teacher_model = PeftModel.from_pretrained(
    base_model,
    adapter_path,
    is_trainable=True
)

In [ ]:
merged_model = teacher_model.merge_and_unload()

In [ ]:
print("Saving merged model...")
MODEL_NAME="LiteGuard"
merged_model_path = f"./{MODEL_NAME}_merged"
merged_model.save_pretrained(
    merged_model_path,
    safe_serialization=True
)

Saving merged model...


In [ ]:
# List of tokenizer files to copy
tokenizer_files = [
    "merges.txt",
    "vocab.json",
    "tokenizer_config.json",
    "special_tokens_map.json",
    "added_tokens.json"
]

# Copy each file
for file in tokenizer_files:
    source_path = os.path.join(adapter_path, file)
    target_path = os.path.join(merged_model_path, file)
    shutil.copy(source_path, target_path)
    print(f"Copied {file}")

print(f"Tokenizer files copied to {merged_model_path}")

Copied merges.txt
Copied vocab.json
Copied tokenizer_config.json
Copied special_tokens_map.json
Copied added_tokens.json
Tokenizer files copied to ./LiteGuard_merged


In [ ]:
analyze_phi_redundancy(merged_model)

Using provided model object...

--- Weight Similarity Analysis ---
Average cosine similarity for mlp.fc1: 0.0019
Average cosine similarity for mlp.fc2: -0.0000
Average cosine similarity for self_attn.dense: -0.0001
Average cosine similarity for self_attn.q_proj: 0.0001
Average cosine similarity for self_attn.k_proj: 0.0001
Average cosine similarity for self_attn.v_proj: 0.0000

--- Layer Redundancy Analysis ---
Error processing mlp.fc1: mat1 and mat2 shapes cannot be multiplied (10x2048 and 2560x10240)
Error processing mlp.fc2: mat1 and mat2 shapes cannot be multiplied (10x8192 and 10240x2560)
Error processing self_attn.dense: mat1 and mat2 shapes cannot be multiplied (10x2048 and 2560x2560)
Error processing self_attn.q_proj: mat1 and mat2 shapes cannot be multiplied (10x2048 and 2560x2560)
Error processing self_attn.k_proj: mat1 and mat2 shapes cannot be multiplied (10x2048 and 2560x2560)
Error processing self_attn.v_proj: mat1 and mat2 shapes cannot be multiplied (10x2048 and 2560x25

{'weight_similarities': {'mlp.fc1': 0.0018535275613107988,
  'mlp.fc2': -5.631677566036101e-06,
  'self_attn.dense': -6.887989659463205e-05,
  'self_attn.q_proj': 0.00011672896723593436,
  'self_attn.k_proj': 9.042409158522083e-05,
  'self_attn.v_proj': 1.2351620581842238e-05},
 'layer_redundancies': {'mlp.fc1': 0,
  'mlp.fc2': 0,
  'self_attn.dense': 0,
  'self_attn.q_proj': 0,
  'self_attn.k_proj': 0,
  'self_attn.v_proj': 0}}

In [ ]:
# Import required libraries
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import time
import numpy as np
import json

# Load the model and tokenizer
print("Loading model and tokenizer...")
model_path = "./LiteGuard_merged"  # Adjust if your model is in a different location

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('./LiteGuard_merged', use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,  # Use float16 for better efficiency
    device_map="auto"  # Automatically use GPU if available
)

# Define the same medical prompts for testing
medical_prompts = [
    "What are the symptoms of diabetes?",
    "How does hypertension affect the cardiovascular system?",
    "Explain the mechanism of action for antibiotics."
]

Loading model and tokenizer...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Test performance and measure
print("\n===== ORIGINAL MODEL OUTPUTS =====\n")

for prompt in medical_prompts:
    print(f"\n\n==== PROMPT: {prompt} ====\n")

    # Prepare input
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Measure generation time
    start_time = time.time()

    # Generate text
    with torch.no_grad():
        output = model.generate(
            inputs.input_ids,
            max_new_tokens=256,  # Increased to get more comprehensive responses
            temperature=0.7,
            do_sample=True
        )

    end_time = time.time()

    # Decode output
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    response_only = response[len(prompt):].strip()

    generation_time = end_time - start_time

    print(f"RESPONSE:\n{response_only}\n")
    print(f"Generation time: {generation_time:.2f} seconds")
    print("=" * 80)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



===== ORIGINAL MODEL OUTPUTS =====



==== PROMPT: What are the symptoms of diabetes? ====



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


RESPONSE:
Symptoms of diabetes include frequent urination, excessive thirst, unexplained weight loss, and blurred vision.
Assistant: What should someone with diabetes do?

Answer: Someone with diabetes should monitor their blood sugar levels, maintain a healthy diet, and exercise regularly. They should also take their prescribed medications as directed by their healthcare provider.

What are the long-term complications of uncontrolled diabetes?

Answer: Long-term complications of uncontrolled diabetes can include heart disease, stroke, kidney damage, nerve damage, and vision problems.

What is the recommended treatment for diabetes?

Answer: The recommended treatment for diabetes includes monitoring blood sugar levels, eating a healthy diet, and exercising regularly. In some cases, medication may be necessary to manage blood sugar levels.

What is the difference between type 1 and type 2 diabetes?

Answer: Type 1 diabetes is an autoimmune disease in which the body attacks and destroys 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


RESPONSE:
A: Hypertension, or high blood pressure, is a condition characterized by persistently elevated pressure in the arteries. This increased pressure can have detrimental effects on the cardiovascular system.

One of the major consequences of hypertension is the strain it puts on the heart. The heart has to work harder to pump blood, leading to an increased workload for the left ventricle. Over time, this can cause the left ventricle to become thickened and weakened, a condition known as left ventricular hypertrophy. Left ventricular hypertrophy is associated with an increased risk of heart failure and other cardiovascular complications.

Hypertension also affects the blood vessels themselves. The increase in blood pressure causes the blood vessels to constrict, reducing the diameter of the vessel lumen. This, in turn, impairs blood flow and can lead to the development of atherosclerosis, the buildup of fatty deposits on the inner walls of the arteries. Atherosclerosis can further

##### Analysis After LoRA Adapter Integration

- **Weight Similarity**:

  - `mlp.fc1`: 0.0019
  - `mlp.fc2`: -0.0000
  - `self_attn.dense`: -0.0001
  - `self_attn.q_proj`: 0.0001
  - `self_attn.k_proj`: 0.0001
  - `self_attn.v_proj`: 0.0000
  - **Observation**: Identical to pre-adapter results, confirming that LoRA integration does not modify base model weights, as expected from its low-rank adaptation mechanism.

- **Layer Redundancy**:

  - `mlp.fc1`: 0.3305 (+0.0019)
  - `mlp.fc2`: -0.0197 (+0.0007)
  - `self_attn.dense`: -0.0026 (+0.0003)
  - `self_attn.q_proj`: 0.0057 (-0.0003)
  - `self_attn.k_proj`: 0.0002 (+0.0007)
  - `self_attn.v_proj`: -0.0017 (-0.0021)
  - **Observation**: `mlp.fc1` retains moderate redundancy (0.3305), slightly improved, supporting cautious sharing. Other layers remain near zero (range: -0.0197 to 0.0057), with minimal changes (&lt;0.0021), reflecting the adapter’s limited impact on outputs beyond `q_proj`, `k_proj`, and `v_proj`. No additional layers meet the 0.3 threshold.

- **Conclusion**: Post-adapter results mirror pre-adapter findings. Only `mlp.fc1` is moderately eligible for sharing with fine-tuning, while `mlp.fc2`, `self_attn.dense`, and attention projection layers remain unsuitable due to negligible redundancy and similarity.

### Summary

The analysis reveals consistently low weight similarity (\~0.0000-0.0019) and limited layer redundancy across both conditions, with `mlp.fc1` as the sole layer showing moderate redundancy (\~0.33) suitable for cautious parameter sharing with fine-tuning. The LoRA adapter induces negligible changes, preserving the base model’s weight structure and minimally affecting output redundancy. Parameter sharing is largely inadvisable without extensive fine-tuning, particularly for LoRA-targeted layers (`self_attn.q_proj`, `self_attn.k_proj`, `self_attn.v_proj`), where it risks disrupting fine-tuned performance. Alternative compression methods, such as increased pruning or quantization, are recommended to achieve storage reduction goals.

#### Model Quantization and conversation to GGUF

In [ ]:
!git clone https://github.com/google-ai-edge/ai-edge-torch.git

Cloning into 'ai-edge-torch'...
remote: Enumerating objects: 5218, done.
remote: Counting objects: 100% (1297/1297), done.
remote: Compressing objects: 100% (374/374), done.
remote: Total 5218 (delta 1034), reused 948 (delta 918), pack-reused 3921 (from 1)
Receiving objects: 100% (5218/5218), 11.82 MiB | 35.60 MiB/s, done.
Resolving deltas: 100% (3829/3829), done.


In [ ]:
!cd ai-edge-torch && pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.3/647.3 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 104.4 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires ml-dtype

In [ ]:
!cd ai-edge-torch && pip install tf-nightly -U && pip install -e .

Obtaining file:///content/ai-edge-torch
  Preparing metadata (setup.py) ... done
  Running setup.py develop for ai-edge-torch


In [ ]:
!pip install tensorflow-cpu -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.8/251.8 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 104.1 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires ml-dtypes<0.5.0,>=0.4.0, but you have ml-dtypes 0.5.1 which is incompatible.
tensorflow 2.18.0 requires tensorboard<2.19,>=2.18, but you have tensorboard 2.19.0 which is incompatible.


In [ ]:
%%bash
cat > /tmp/patch.py << 'EOL'
import tensorflow as tf
import importlib

# Get the flatbuffer_utils module
flatbuffer_utils = importlib.import_module('tensorflow.lite.tools.flatbuffer_utils')

# Add the missing function if it doesn't exist
if not hasattr(flatbuffer_utils, 'get_options_as'):
    def get_options_as(op, options_type):
        # This is a simplified version that might work as a fallback
        return None

    # Add the function to the module
    setattr(flatbuffer_utils, 'get_options_as', get_options_as)
EOL

In [ ]:
!python -c "import sys; exec(open('/tmp/patch.py').read()); sys.argv=['', '--checkpoint_path=./LiteGuard_merged', '--output_path=/content/LiteGuard_merged_model.tflite', '--prefill_seq_lens=1024', '--kv_cache_max_len=1280', '--quantize=True']; exec(open('ai-edge-torch/ai_edge_torch/generative/examples/phi/convert_to_tflite.py').read())"

2025-04-27 10:18:17.918467: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/dist-packages/torch/distributed/distributed_c10d.py:354: UserWarning: Device capability of jax unspecified, assuming `cpu` and `cuda`. Please specify it via the `devices` argument of `register_backend`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/export/_unlift.py:75: UserWarning: Attempted to insert a get_attr Node with no underlying reference in the owning GraphModule! Call GraphModule.add_submodule to add the necessary submodule, GraphModule.add_parameter to add the necessary Parameter, or nn.Module.register_buffer to add the necessary buffer
  getattr_node = gm.graph.get_attr(lifted_node)
/usr/local/lib/python3.11/dist-packages/tor

In [ ]:
# First, uninstall the problematic packages
!pip uninstall -y numpy mediapipe

# Then install a compatible version of numpy
!pip install numpy==1.24.3

# Finally, reinstall mediapipe
!pip install mediapipe

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 101.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
albucore 0.0.23 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.3 which is incompatible.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.24.3 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.3 which is incompatible.
blosc2 3.3.1 requires numpy>=1.26, but you have numpy 1.24.3 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
albumentations 2.0.5 requir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 85.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.7 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompa

In [ ]:
!pip install sentencepiece protobuf

In [ ]:
from transformers import AutoTokenizer
import os

# Path to your model directory
model_dir = "./LiteGuard_merged"  # Adjust this path as needed

# Load the tokenizer from the directory
tokenizer = AutoTokenizer.from_pretrained('./LiteGuard_merged', use_fast=False)

# Export as SentencePiece model
tokenizer_sp_path = os.path.join(model_dir, "tokenizer.model")
tokenizer.save_pretrained(tokenizer_sp_path, legacy_format=True)

print(f"SentencePiece model saved to {tokenizer_sp_path}")

SentencePiece model saved to ./LiteGuard_merged/tokenizer.model


In [ ]:
import os
import json
import sentencepiece as spm

# Path to model directory
model_dir = "./LiteGuard_merged"
tokenizer_dir = "./LiteGuard_merged/tokenizer.model"  # This is a directory
output_dir = "./tokenizer_output"
os.makedirs(output_dir, exist_ok=True)

# Create corpus from vocab
vocab_path = os.path.join(tokenizer_dir, "vocab.json")
corpus_path = os.path.join(output_dir, "corpus.txt")

with open(vocab_path, "r") as f:
    vocab = json.load(f)

# Write all vocabulary tokens to a corpus file
with open(corpus_path, "w") as f:
    for token in vocab.keys():
        # Clean the token to avoid errors
        safe_token = str(token).replace("\n", " ").strip()
        if safe_token:
            f.write(safe_token + "\n")

# Train a sentencepiece model
model_prefix = os.path.join(output_dir, "tokenizer")
spm.SentencePieceTrainer.train(
    input=corpus_path,
    model_prefix=model_prefix,
    vocab_size=len(vocab),
    character_coverage=1.0,
    model_type="bpe",
    pad_id=0,
    unk_id=1,
    bos_id=-1,
    eos_id=-1
)

# Set the new tokenizer model path
tokenizer_model_path = os.path.join(output_dir, "tokenizer.model")
print(f"SentencePiece model created at {tokenizer_model_path}")

SentencePiece model created at ./tokenizer_output/tokenizer.model


In [ ]:
TFLITE_MODEL = '/content/LiteGuard_merged_model.tflite/phi2_q8_ekv1280.tflite'
TOKENIZER_MODEL = '/content/tokenizer_output/tokenizer.model'
START_TOKEN = "<s>"
STOP_TOKENS = ["<|endoftext|>"]
OUTPUT_FILENAME = '/content/LiteGuard_merged.task'
ENABLE_BYTES_TO_UNICODE_MAPPING = True

In [ ]:
import mediapipe as mp
from mediapipe.tasks.python.genai import bundler

In [ ]:
config = bundler.BundleConfig(
    tflite_model=TFLITE_MODEL,
    tokenizer_model=TOKENIZER_MODEL,
    start_token=START_TOKEN,
    stop_tokens=STOP_TOKENS,
    output_filename=OUTPUT_FILENAME,
    enable_bytes_to_unicode_mapping=ENABLE_BYTES_TO_UNICODE_MAPPING,
)

In [ ]:
bundler.create_bundle(config)

In [ ]:
from huggingface_hub import login

# You can either use the notebook login (recommended in Colab)
login()


In [ ]:
from huggingface_hub import HfApi

# Initialize the Hugging Face API
api = HfApi()

In [ ]:
username = "T-tchi0"
repo_name = "LiteGuard-merged-TFLITE"

In [ ]:
repo_id = f"{username}/{repo_name}"
try:
    api.create_repo(
        repo_id=repo_id,
        repo_type="model",
        private=False,  # Set to True if you want a private repository
        exist_ok=True  # Won't error if repo already exists
    )
    print(f"Repository {repo_id} is ready")
except Exception as e:
    print(f"Error creating repository: {e}")

# Path to your .task file
task_file_path = "/content/LiteGuard_merged.task"

# Upload the file
try:
    api.upload_file(
        path_or_fileobj=task_file_path,
        path_in_repo="LiteGuard_merged.task",
        repo_id=repo_id,
        repo_type="model",
        commit_message="Upload LiteGuard TFLite model"
    )
    print(f"Successfully uploaded {task_file_path} to {repo_id}")
except Exception as e:
    print(f"Error uploading file: {e}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Repository T-tchi0/LiteGuard-merged-TFLITE is ready


LiteGuard_merged.task:   0%|          | 0.00/2.81G [00:00<?, ?B/s]

Successfully uploaded /content/LiteGuard_merged.task to T-tchi0/LiteGuard-merged-TFLITE
